In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import html
from urllib.parse import urljoin
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC  
warnings.filterwarnings('ignore')
options=Options()

options.add_extension('../driver/adblock.crx')
options.add_argument('--start-maximized')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--incognito')
#options.add_argument('user-data-dir=selenium')

In [2]:
PATH = ChromeDriverManager().install()

In [3]:
driver = webdriver.Chrome(PATH, options=options)

In [4]:
url = 'https://www.moto1pro.com/'

In [5]:
driver.get(url)

In [6]:
# Acepto las cookies
cookies = driver.find_element(By.XPATH, '/html/body/div[11]/div[2]/div[1]/div[2]/div[2]/button[1]/p')
cookies.click()
time.sleep(0.5)

In [7]:
#Accedo a las fichas tecnicas
fichas = driver.find_elements(By.XPATH, '//*[@id="navWrapper"]/li[6]/a')
fichas[0].click()
time.sleep(0.5)

In [8]:
marcas_names = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div[1]')
#marcas_names

In [9]:
string = marcas_names[0].text

In [10]:
marcas_list = string.split('\n')

In [11]:
marcas_list = [marca.title() for marca in marcas_list]
#marcas_list

In [12]:
elems = driver.find_elements(By.XPATH, "//a[@href]")

In [13]:
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [14]:
links = [elem.get_attribute("href") for elem in elems]

In [15]:
links_marcas = []              # Aquí me da todos los links de todas las marcas disponibles en la web
for e in links:
    if 'marca-de-motos/' in e:
        links_marcas.append(e)
#links_marcas

In [16]:
url_base = "https://www.moto1pro.com/marca-de-motos/"                                    # Aquí filtro por las marcas mas vendidas y compradas pero da links duplicados
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

full_links = []                                                           
visited_links = set()

for marca in marcas:
    url = urljoin(url_base, marca)
    response = requests.get(url)
    tree = html.fromstring(response.content)
    div_content = tree.xpath('//div[@class="contenido-izquierda"]')[0]
    links = div_content.xpath('.//a/@href')
    for link in links:
        full_link = urljoin(url, link)
        if full_link not in visited_links:
            full_links.append(full_link)
            visited_links.add(full_link)
full_links

['https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SR-GT-125-2022',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuono-660-Factory-2022',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuareg-660-Evocative-2022',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuareg-660-2022',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-RX-125-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SX-125-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SXR-50-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-RS-125-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuono-125-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuono-V4-Factory-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-RSV-4-1100-Factory-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuono-660-A2-2021',
 'https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuono-660-2021'

In [17]:
columns_filtering = ['Marca de la moto',
 'Modelo',
 'Precio de la moto',
 'Tipo de Carné',
 'Tipo de moto',
 'Versión / Año',
 'Web del fabricante',
 'Cilindrada',
 'Potencia Máxima Declarada',
 'Altura asiento']

In [18]:
dfs = [] 
for link in full_links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    data = {}
    rows = soup.find_all('tr')

    try:
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                data[key] = value
        df = pd.DataFrame.from_records([data])
        df_filtered = df[columns_filtering]
        dfs.append(df_filtered)
    except:
        print(f"No se pudieron obtener los datos para el enlace: {link}")

No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SX-125-2018
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-RX-125-2018
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SX-50-Factory-2018
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-RX-50-Factory-2018
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SR-50-Motard-2018
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-SR-50-R-2018
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Tuono-125-2017
No se pudieron obtener los datos para el enlace: https://www.moto1pro.com/marca-de-motos/Aprilia/Aprilia-Shiver-900-2017
No se pudieron obtener los datos par

In [19]:
df_final = pd.concat(dfs, ignore_index=True)
pd.set_option('display.max_columns', None)

In [20]:
df_final

,Marca de la moto,Modelo,Precio de la moto,Tipo de Carné,Tipo de moto,Versión / Año,Web del fabricante,Cilindrada,Potencia Máxima Declarada,Altura asiento
0,Aprilia,SR GT 125 2022,3.999 €,A1,Scooter,2022,www.aprilia.com,124 cc,15 cv,799 mm
1,Aprilia,Tuono 660 Factory 2022,10.999 €,A,Naked,2022,www.aprilia.com/es_ES/,659 cc,100 cv,820 mm
2,Aprilia,Tuareg 660 Evocative 2022,12.699 €,A,Trail,2022,www.aprilia.com/es_ES/,659 cc,80 cv,860 mm
3,Aprilia,Tuareg 660 2022,11.999 €,A,Trail,2022,www.aprilia.com/es_ES/,659 cc,80 cv,860 mm
4,Aprilia,RX 125 2021,3.999 €,A1,Naked,2021,http://www.aprilia.com/es_ES/,124 cc,15 cv,880 mm
...,...,...,...,...,...,...,...,...,...,...
628,KTM,1290 Super Adventure,17.953 €,A,Trail,2015,www.ktm.com,1.301 cc,160 cv,860 mm
629,KTM,1290 Super Duke GT,18.586 €,A,Touring,2016,www.ktm.com,1.295 cc,173 cv,835 mm
630,KTM,1290 Super Duke R,16.318 €,A,Naked,2014,www.ktm.com,1.295 cc,180 cv,835 mm
631,KTM,690 SMC R,8.699 €,A2,Offroad,2015,http://www.ktm.com,690 cc,67 cv,890 mm
